In [1]:
import pandas as pd
import numpy as np
data_rfe=pd.read_csv('norm_jinshi.csv')
data_rfe=np.array(data_rfe)
fea=data_rfe[:,:49]
target=data_rfe[:,49]
x_test=fea[:170,:]
y_test=target[:170]
x_train=fea[170:,:]
y_train=target[170:]

# SVM-RFE_RELIEF

In [2]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,alpha,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,self.X_train,self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            #print(relief_score)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            #print(rfe_score)
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            #print(total_score)
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:,candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
    def sele_feature(self,alpha,optim_featur_num):
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        while True:
            relief_score=(reliefF.reliefF(self.X_train[:,candi_fea],self.Y_train)).reshape(1,i)
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            total_score=alpha*relief_score+(1-alpha)*rfe_score
            index_=np.argmin(total_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            if len(candi_fea)==optim_featur_num:
                return candi_fea

In [3]:
rfe_relief_model=rfe_relief_SVM(x_train,y_train,1.5)

In [4]:
aver_mean_lis1,F_measure_lis1=rfe_relief_model.featu_score(0.15,5)
print(aver_mean_lis1,F_measure_lis1)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8106007620439479, 0.8100905579623154, 0.809075369278146, 0.8157119369486925, 0.8167349548515059, 0.8162234459000992, 0.8131596116707552, 0.813671120622162, 0.8152069523461558, 0.8141852393131165, 0.813669815752388, 0.8126494075891225, 0.8141761052246986, 0.8146928336552012, 0.8197974842110757, 0.815191293908868, 0.8105929328253041, 0.8162208361605512, 0.8136685108826139, 0.8146941385249752, 0.8106020669137219, 0.8085560311080954, 0.8095777441411348, 0.8131530873218853, 0.8121326791586199, 0.8095738295318127, 0.8095738295318128, 0.8085508116289993, 0.809054491361762, 0.8105903230857561, 0.8105864084764342, 0.8110979174278408, 0.8131465629730152, 0.8105942376950781, 0.8111044417767108, 0.8065113001722428, 0.8075343180750562, 0.8075356229448302, 0.8080458270264627, 0.8080497416357847, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0.6

In [5]:
aver_mean_lis2,F_measure_lis2=rfe_relief_model.featu_score(0.25,5)
print(aver_mean_lis2,F_measure_lis2)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8106007620439479, 0.8100905579623154, 0.809075369278146, 0.8157119369486925, 0.8167349548515059, 0.8162234459000992, 0.8131596116707552, 0.813671120622162, 0.8152069523461558, 0.8141852393131165, 0.813669815752388, 0.8126494075891225, 0.8141761052246986, 0.8146928336552012, 0.8197974842110757, 0.815191293908868, 0.8141656662665067, 0.8172347199749463, 0.8116146458583433, 0.8126350540216085, 0.812123545070202, 0.8111096612558066, 0.8126441881100266, 0.8126389686309305, 0.8131491727125633, 0.8146797849574613, 0.8192755363014772, 0.8095660003131687, 0.8105916279555301, 0.8105903230857561, 0.8126376637611565, 0.8110979174278408, 0.8131465629730152, 0.8105942376950781, 0.8105981523044001, 0.8136632914035179, 0.8075343180750562, 0.8075356229448302, 0.8080458270264627, 0.8080497416357847, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0.6

In [6]:
aver_mean_lis3,F_measure_lis3=rfe_relief_model.featu_score(0.35,5)
print(aver_mean_lis3,F_measure_lis3)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8106007620439479, 0.8100905579623154, 0.809075369278146, 0.8157119369486925, 0.8136685108826139, 0.813669815752388, 0.8141839344433425, 0.8152082572159299, 0.8111187953442247, 0.8126559319379926, 0.812143118116812, 0.8101023017902813, 0.8126533221984447, 0.8121405083772639, 0.8126428832402526, 0.8141748003549246, 0.8141721906153766, 0.8157028028602745, 0.8157054125998225, 0.8146797849574613, 0.8172373297144946, 0.8151952085181898, 0.8162182264210032, 0.8116211702072134, 0.8131491727125633, 0.8167219061537659, 0.8192755363014772, 0.8095660003131687, 0.8105916279555301, 0.8085521164987736, 0.8126376637611565, 0.8110979174278408, 0.8105903230857561, 0.8085455921499035, 0.8100801190041234, 0.8095751344015867, 0.8085508116289993, 0.8075356229448302, 0.8080458270264627, 0.8080497416357847, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0

In [7]:
aver_mean_lis4,F_measure_lis4=rfe_relief_model.featu_score(0.45,5)
print(aver_mean_lis4,F_measure_lis4)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8080510465055587, 0.8126402735007046, 0.809075369278146, 0.8157119369486925, 0.8177540581449971, 0.8131583068009813, 0.8121378986377159, 0.8100983871809593, 0.8131635262800773, 0.8152095620857038, 0.8141865441828905, 0.812144422986586, 0.8090766741479201, 0.8111174904744507, 0.8116211702072134, 0.8131556970614333, 0.8146850044365573, 0.8172347199749463, 0.8177449240565793, 0.8141669711362807, 0.8151899890390938, 0.8167284305026359, 0.8157041077300488, 0.8131517824521113, 0.8126402735007046, 0.8167219061537659, 0.8192755363014772, 0.8095660003131687, 0.8105916279555301, 0.8085521164987736, 0.8126376637611565, 0.8121300694190718, 0.8105955425648521, 0.8116185604676653, 0.8090662351897281, 0.8039628895036277, 0.8085508116289993, 0.8075356229448302, 0.8080458270264627, 0.8080497416357847, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 

In [8]:
aver_mean_lis5,F_measure_lis5=rfe_relief_model.featu_score(0.55,5)
print(aver_mean_lis5,F_measure_lis5)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8080510465055587, 0.8126402735007046, 0.809075369278146, 0.8157119369486925, 0.8167310402421837, 0.8095855733597788, 0.8116276945560832, 0.8121378986377159, 0.8111201002139985, 0.8146980531342972, 0.812143118116812, 0.8101023017902813, 0.8126533221984447, 0.8126546270682187, 0.8162247507698732, 0.8131570019312072, 0.8121339840283941, 0.8136645962732919, 0.8177449240565793, 0.8141669711362807, 0.8151899890390938, 0.8167284305026359, 0.8157041077300488, 0.8177475337961273, 0.8177488386659011, 0.8187679419593925, 0.8203011639438383, 0.8116172555978913, 0.8146836995667833, 0.8085521164987736, 0.8126376637611565, 0.8121300694190718, 0.8105955425648521, 0.8116185604676653, 0.8090662351897281, 0.8039628895036277, 0.8060037058301581, 0.8075356229448302, 0.8080458270264627, 0.8080497416357847, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 

In [9]:
aver_mean_lis6,F_measure_lis6=rfe_relief_model.featu_score(0.65,5)
print(aver_mean_lis6,F_measure_lis6)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8080510465055587, 0.8126402735007046, 0.8167388694608277, 0.8157119369486925, 0.8177540581449971, 0.8136672060128399, 0.8116276945560832, 0.8121378986377159, 0.8111201002139985, 0.8146980531342972, 0.812143118116812, 0.810608591262592, 0.812144422986586, 0.8080549611148807, 0.8100944725716375, 0.8126467978495746, 0.8146863093063313, 0.8151978182577378, 0.8141734954851504, 0.8141669711362807, 0.8095699149224908, 0.8146850044365573, 0.8105955425648521, 0.8151912939088678, 0.81366068166397, 0.8126389686309308, 0.8157067174695966, 0.8162195312907772, 0.8187692468291665, 0.8157041077300484, 0.8126376637611565, 0.8121300694190718, 0.8105955425648521, 0.8116185604676653, 0.8090662351897281, 0.8060050106999321, 0.8070280286027455, 0.8085573359778694, 0.8085586408476434, 0.8080471318962367, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0.6

In [10]:
aver_mean_lis7,F_measure_lis7=rfe_relief_model.featu_score(0.75,5)
print(aver_mean_lis7,F_measure_lis7)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8080510465055587, 0.8126402735007046, 0.8167388694608277, 0.8167336499817317, 0.8177540581449971, 0.8136672060128399, 0.8116276945560832, 0.8121378986377159, 0.8111201002139985, 0.8146980531342972, 0.812143118116812, 0.810608591262592, 0.812144422986586, 0.8080549611148807, 0.8100944725716375, 0.8126467978495746, 0.8146863093063313, 0.8151978182577378, 0.8141734954851504, 0.8131543921916593, 0.8116172555978913, 0.8136619865337439, 0.8095764392713607, 0.8105968474346261, 0.8054908920089776, 0.8095764392713607, 0.8080445221566889, 0.8136619865337439, 0.8126389686309308, 0.8146823946970091, 0.8141708857456026, 0.8116198653374391, 0.8121326791586199, 0.8095816587504567, 0.8080523513753327, 0.8080484367660107, 0.8075382326843782, 0.8085573359778694, 0.8085586408476434, 0.8080471318962367, 0.8080484367660107, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0

In [11]:
aver_mean_lis8,F_measure_lis8=rfe_relief_model.featu_score(0.85,5)
print(aver_mean_lis8,F_measure_lis8)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.7927631922334151, 0.798859543817527, 0.8080497416357847, 0.8111135758651287, 0.8116250848165354, 0.8136632914035179, 0.8167336499817317, 0.8162273605094212, 0.8136672060128399, 0.8126494075891225, 0.8095868782295528, 0.8126559319379926, 0.8126559319379926, 0.8111201002139987, 0.810607286392818, 0.812144422986586, 0.8080549611148807, 0.8100944725716375, 0.8121378986377159, 0.8162195312907772, 0.8177488386659013, 0.8187705516989405, 0.8146876141761054, 0.8141800198340207, 0.8157106320789185, 0.8146876141761051, 0.8187705516989405, 0.8151978182577378, 0.8151965133879641, 0.8131517824521113, 0.8100892530925414, 0.8111148807349027, 0.812128764549298, 0.8116211702072134, 0.8111122709953547, 0.8106007620439479, 0.8095816587504567, 0.8080523513753327, 0.8080510465055589, 0.8075382326843782, 0.804987212276215, 0.801924682916645, 0.8009003601440575, 0.8044743984550342, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0.6

In [12]:
aver_mean_lis9,F_measure_lis9=rfe_relief_model.featu_score(0.95,5)
print(aver_mean_lis9,F_measure_lis9)

[0.49974555039407065, 0.5808719139829844, 0.7927618873636411, 0.795815282634793, 0.7958165875045671, 0.800391460932199, 0.8116237799467614, 0.8126454929798006, 0.8100905579623154, 0.8111122709953544, 0.8162273605094212, 0.8146954433947492, 0.8121418132470382, 0.8111266245628685, 0.8111240148233207, 0.8060128399185761, 0.8090818936270161, 0.8090779790176941, 0.8090779790176941, 0.8085638603267394, 0.8100944725716375, 0.8121378986377159, 0.8152056474763819, 0.8141800198340204, 0.8146902239156532, 0.8136724254919357, 0.8131635262800773, 0.8157171564277885, 0.8162208361605512, 0.8126454929798006, 0.813672425491936, 0.810608591262592, 0.807540842423926, 0.8060037058301581, 0.8054935017485254, 0.8060076204394802, 0.8095803538806827, 0.8085560311080954, 0.8080458270264627, 0.8049846025366669, 0.8070306383422935, 0.8070306383422935, 0.8075382326843782, 0.804987212276215, 0.801924682916645, 0.8009003601440575, 0.8044743984550342, 0.8008938357951877, 0.8019103293491309] [0.5173615245343753, 0.68

In [24]:
np.max(aver_mean_lis1[:10]),np.max(aver_mean_lis2[:10]),np.max(aver_mean_lis3[:10]),np.max(aver_mean_lis4[:10]),np.max(aver_mean_lis5[:10])

(0.8157119369486925,
 0.8157119369486925,
 0.8157119369486925,
 0.8157119369486925,
 0.8157119369486925)

In [25]:
np.max(aver_mean_lis6[:10]),np.max(aver_mean_lis7[:10]),np.max(aver_mean_lis8[:10]),np.max(aver_mean_lis9[:10])

(0.8167388694608277,
 0.8167388694608277,
 0.8167336499817317,
 0.8126454929798006)

In [26]:
np.argmax(aver_mean_lis6[:10]),np.argmax(aver_mean_lis7[:10]),np.argmax(aver_mean_lis8[:10])

(8, 8, 9)

In [27]:
aver_mean_lis6[:10]

[0.49974555039407065,
 0.5808719139829844,
 0.7927618873636411,
 0.7927631922334151,
 0.798859543817527,
 0.8080497416357847,
 0.8080510465055587,
 0.8126402735007046,
 0.8167388694608277,
 0.8157119369486925]

# SVM-RFE

In [14]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class rfe_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        i=featu_num
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        aver=mean(cross_val_score(estimator,x_train,y_train,scoring="accuracy",cv=kf,n_jobs=-1))
        aver_lis[i-1]=aver
        F1=mean(cross_val_score(estimator,x_train,y_train,scoring="f1",cv=kf,n_jobs=-1))
        F_lis[i-1]=F1
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            i-=1
            aver=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,x_train[:,candi_fea],y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i-1]=aver
            F_lis[i-1]=F1
            if i==1:
                break
        return aver_lis,F_lis
   
    def featu_subset(self,featur_num):
        kf=KFold(n_splits=5,shuffle=False)
        estimator=SVC(kernel='linear',C=self.C)
        rank_=[]
        featu_num=self.X_train.shape[1]
        candi_fea=[i for i in range(featu_num)]
        while True:
            estimator.fit(self.X_train[:,candi_fea],self.Y_train)
            rfe_score=(estimator.coef_)**2
            index_=np.argmin(rfe_score)
            remove_fea=candi_fea[index_]
            candi_fea.remove(remove_fea)
            if len(candi_fea)==featur_num:
                break
        return candi_fea

In [15]:
rfe_SVM_model=rfe_SVM(x_train,y_train,1.5)
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis=rfe_SVM_model.featu_score(5)

In [16]:
rfe_SVM_aver_mean_lis,rfe_SVM_F_measure_lis

([0.49974555039407065,
  0.5808719139829844,
  0.7927618873636411,
  0.7927631922334151,
  0.798859543817527,
  0.8080497416357847,
  0.8106007620439479,
  0.8100905579623154,
  0.809075369278146,
  0.8157119369486925,
  0.8167349548515059,
  0.8162234459000992,
  0.8131596116707552,
  0.813671120622162,
  0.8152069523461558,
  0.8141852393131165,
  0.813669815752388,
  0.8126494075891225,
  0.8141761052246986,
  0.8146928336552012,
  0.8197974842110757,
  0.815191293908868,
  0.8105929328253041,
  0.8116120361187953,
  0.8116211702072134,
  0.8131517824521113,
  0.8105981523044001,
  0.8157080223393706,
  0.8136619865337439,
  0.8121300694190718,
  0.8167232110235398,
  0.8146823946970091,
  0.815189989039094,
  0.8131478678427893,
  0.8167192964142179,
  0.813658071924422,
  0.8100879482227674,
  0.8136645962732919,
  0.8136645962732919,
  0.8105981523044001,
  0.8070254188631975,
  0.8065126050420168,
  0.8085586408476434,
  0.8065126050420168,
  0.8065086904326948,
  0.803955060284

# SVM-RELIEF

In [17]:
from skfeature.function.similarity_based import reliefF
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from numpy import mean 
from sklearn.model_selection import cross_val_score
class relief_SVM:
    def __init__(self,x_train,y_train,C):
        self.X_train=x_train
        self.Y_train=y_train
        self.C=C
    def featu_score(self,folds):
        kf=KFold(n_splits=folds,shuffle=False)
        featu_num=self.X_train.shape[1]
        relief_fea_score=reliefF.reliefF(self.X_train,self.Y_train)
        relief_candi_fea=list(reliefF.feature_ranking(relief_fea_score))
        aver_lis=[i for i in range(featu_num)]
        F_lis=[i for i in range(featu_num)]
        for i in range(featu_num):
            estimator=SVC(kernel='linear',C=self.C)
            candi_fea=relief_candi_fea[:(i+1)]
            aver=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="accuracy",cv=kf,n_jobs=-1))
            F1=mean(cross_val_score(estimator,self.X_train[:, candi_fea],self.Y_train,scoring="f1",cv=kf,n_jobs=-1))
            aver_lis[i]=aver
            F_lis[i]=F1 
        return aver_lis,F_lis

In [18]:
relief_SVM_model=relief_SVM(x_train,y_train,1.5)
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis=relief_SVM_model.featu_score(5)

In [19]:
relief_SVM_aver_mean_lis,relief_SVM_F_measure_lis

([0.7896993580040712,
  0.7973393705308209,
  0.798867373036171,
  0.7993736625084817,
  0.798863458426849,
  0.7942624876037371,
  0.7957957095881831,
  0.7937496737825565,
  0.8100892530925414,
  0.8100918628320894,
  0.80907014979905,
  0.807543452163474,
  0.8065165196513387,
  0.8106020669137219,
  0.8167258207630879,
  0.8162182264210032,
  0.8157080223393706,
  0.8136632914035179,
  0.8100879482227674,
  0.8167232110235398,
  0.8162143118116812,
  0.815189989039094,
  0.8141708857456026,
  0.8111057466464848,
  0.8111005271673886,
  0.8100801190041234,
  0.812124849939976,
  0.8110979174278408,
  0.8065099953024688,
  0.8177410094472572,
  0.8157067174695966,
  0.8167258207630879,
  0.8167245158933139,
  0.8131543921916593,
  0.8100879482227674,
  0.8126402735007046,
  0.8116198653374394,
  0.8095790490109088,
  0.8060115350488021,
  0.8070358578213893,
  0.8044743984550342,
  0.8044757033248082,
  0.8070188945143274,
  0.8080419124171406,
  0.8039641943734015,
  0.8080471318962

In [20]:
jinshi_accuracy=[]
jinshi_accuracy.append(aver_mean_lis1)
jinshi_accuracy.append(aver_mean_lis2)
jinshi_accuracy.append(aver_mean_lis3)
jinshi_accuracy.append(aver_mean_lis4)
jinshi_accuracy.append(aver_mean_lis5)
jinshi_accuracy.append(aver_mean_lis6)
jinshi_accuracy.append(aver_mean_lis7)
jinshi_accuracy.append(aver_mean_lis8)
jinshi_accuracy.append(aver_mean_lis9)
jinshi_accuracy.append(rfe_SVM_aver_mean_lis)
jinshi_accuracy.append(relief_SVM_aver_mean_lis)

In [21]:
colu=[i for i in range(1,50)]
jinshi_acurracy_dataframe=pd.DataFrame(columns=colu,data=np.array(jinshi_accuracy))
jinshi_acurracy_dataframe.to_csv("jinshi_accuracy.csv")

In [22]:
jinshi_F_measure=[]
jinshi_F_measure.append(F_measure_lis1)
jinshi_F_measure.append(F_measure_lis2)
jinshi_F_measure.append(F_measure_lis3)
jinshi_F_measure.append(F_measure_lis4)
jinshi_F_measure.append(F_measure_lis5)
jinshi_F_measure.append(F_measure_lis6)
jinshi_F_measure.append(F_measure_lis7)
jinshi_F_measure.append(F_measure_lis8)
jinshi_F_measure.append(F_measure_lis9)
jinshi_F_measure.append(rfe_SVM_F_measure_lis)
jinshi_F_measure.append(relief_SVM_F_measure_lis)

In [23]:
colu=[i for i in range(1,50)]
jinshi_F_measure_dataframe=pd.DataFrame(columns=colu,data=np.array(jinshi_F_measure))
jinshi_F_measure_dataframe.to_csv("jinshi_F_measure.csv")